In [53]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

First, let's get the html from the Wikipedia page as lxml.

In [20]:
page = requests.get(r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')             
soup = BeautifulSoup(page.text,features='lxml')

Append all of the rows from the wiki table to an array, then turn it into a dataframe.

In [118]:
df = []

for row in soup.find_all('tr'):
    if len(row.find_all('td')) == 3:
        df.append([val.text.replace("\n","") for val in row.find_all('td')])
        
df = pd.DataFrame(df, columns = ['Postcode','Borough','Neighborhood'])
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Remove all of the rows with a borough of "Not assigned".  
Also, "Not assigned" neighborhoods will be the same as their respective boroughs.

In [119]:
df = df[df['Borough']!="Not assigned"]
df.loc[df['Neighborhood']=="Not assigned","Neighborhood"] = df[df['Neighborhood']=="Not assigned"]["Borough"]

Postcodes can have multiple neighborhoods, so we combine them.

In [120]:
df = df.groupby(by=['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [121]:
df.shape

(103, 3)